<a href="https://colab.research.google.com/github/ss8319/FILM/blob/main/Shamus_FILM_frame_interpolation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title セットアップ

# githubからコード取得 & ライブラリ・インストール
! git clone https://github.com/cedro3/frame-interpolation.git
%cd frame-interpolation
! pip install -r requirements.txt

# 学習済みパラメータ・ダウンロード
! pip install --upgrade gdown
import gdown
gdown.download('https://drive.google.com/uc?id=1rEABCoyQFkmHGieKDhHXW2ZYJi12lofI', 'pretrained_models.zip', quiet=False)
! unzip pretrained_models.zip

# ライブラリ・インポート
from function import *

In [ ]:
#@title 補間動画の作成
select_folder = 'photo_2'#@param {type:"string"}
times_to_interpolate =  6#@param {type:"integer"}

! python -m eval.interpolator_cli \
     --pattern $select_folder\
     --model_path pretrained_models/film_net/Style/saved_model \
     --times_to_interpolate $times_to_interpolate \
     --output_video


In [ ]:
#@title 動画の再生
display_mp4(select_folder+'/interpolated.mp4')

In [ ]:
#@title 前後付き動画に編集
import cv2
import glob

reset_folder(select_folder+'/images_t')
reset_folder(select_folder+'/images_y')

files = sorted(glob.glob(select_folder+'/interpolated_frames/*.png'))
img_1 = cv2.imread(select_folder+'/frame1.jpg')
img_2 = cv2.imread(select_folder+'/frame2.jpg')
img_1 = cv2.resize(img_1, dsize=None, fx=0.5, fy=0.5)
img_2 = cv2.resize(img_2, dsize=None, fx=0.5, fy=0.5)

for file in files:
    img = cv2.imread(file)

    # 縦長
    up = cv2.hconcat([img_1, img_2])
    image1 = cv2.vconcat([up, img])
    name = os.path.basename(file)
    cv2.imwrite(select_folder+'/images_t/'+name, image1)

    # 横長
    side = cv2.vconcat([img_1, img_2])
    image2 = cv2.hconcat([side, img])
    name = os.path.basename(file)
    cv2.imwrite(select_folder+'/images_y/'+name, image2)

!ffmpeg -y -r 30 -i $select_folder/images_t/frame_%03d.png -vcodec libx264 -pix_fmt yuv420p -loglevel error output1.mp4
!ffmpeg -y -r 30 -i $select_folder/images_y/frame_%03d.png -vcodec libx264 -pix_fmt yuv420p -loglevel error output2.mp4


In [ ]:
#@title  横に付加した動画の再生
display_mp4('output2.mp4')

In [ ]:
#@title 上に付加した動画の再生
display_mp4('output1.mp4')